In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

# Process data

In [9]:
def get_data():
    test_dfn = pd.read_csv(r'test_df_after_fs_100.csv')
    #test_dfn = test_dfn.drop(['id'], axis=1)
    train_dfn = pd.read_csv(r'train_df_after_fs_100.csv')
    #train_dfn = train_dfn.drop(['id'], axis=1)
    
    X_train_df = train_dfn.iloc[:, :train_dfn.shape[1]-1]
    X_test_df = test_dfn.iloc[:, :test_dfn.shape[1]-1]

    Y_train_df = train_dfn.iloc[:, -1:]
    Y_test_df = test_dfn.iloc[:, -1:]

    
    return X_train_df, Y_train_df, X_test_df, Y_test_df


X_train_df, Y_train_df, X_test_df, Y_test_df = get_data()

In [11]:
pip install optuna

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [13]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

import seaborn as sns
import matplotlib.pyplot as plt

# KNN Tuning

In [15]:
import optuna
from sklearn.neighbors import KNeighborsClassifier




def knn_objective(trial):

    optimizer = trial.suggest_categorical('algorithm', ['auto','ball_tree','kd_tree','brute'])
    rf_max_depth = trial.suggest_int("k_n_neighbors", 2, 10, log=True)
    weights = trial.suggest_categorical("weights", ["uniform",  "distance"])
    p = trial.suggest_categorical("p", [1, 2])
    leaf_size = trial.suggest_int("leaf_size", 2, 16, step=4),
    
        
    clf = KNeighborsClassifier(n_neighbors=rf_max_depth,algorithm=optimizer, weights = weights, p = p, n_jobs = -1)
    clf.fit(X_train_df, Y_train_df)

    preds = clf.predict(X_test_df)
    pred_labels = np.rint(preds)
    accuracy = metrics.accuracy_score(Y_test_df, pred_labels)
    accuracy

    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(knn_objective, n_trials=128, timeout=360000)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-02-06 16:41:33,699] A new study created in memory with name: no-name-5c5bd6e2-1f2a-4f27-a479-b9cea69b943f
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
[I 2023-02-06 16:47:13,051] Trial 0 finished with value: 0.8000709754691034 and parameters: {'algorithm': 'auto', 'k_n_neighbors': 4, 'weights': 'distance', 'p': 1, 'leaf_size': 14}. Best is trial 0 with value: 0.8000709754691034.
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution i

[I 2023-02-06 17:22:04,223] Trial 10 finished with value: 0.8124916825622145 and parameters: {'algorithm': 'auto', 'k_n_neighbors': 10, 'weights': 'distance', 'p': 2, 'leaf_size': 2}. Best is trial 7 with value: 0.8221620902275651.
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
[I 2023-02-06 17:22:36,849] Trial 11 finished with value: 0.8225169675730826 and parameters: {'algorithm': 'ball_tree', 'k_n_neighbors': 10, 'weights': 'uniform', 'p': 2, 'leaf_size': 6}. Best is trial 11 with value: 0.8225169675730826.
C:\Users

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
[I 2023-02-06 17:35:53,992] Trial 21 finished with value: 0.8225613272412723 and parameters: {'algorithm': 'brute', 'k_n_neighbors': 10, 'weights': 'uniform', 'p': 2, 'leaf_size': 2}. Best is trial 20 with value: 0.8225613272412723.
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  re

C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
[I 2023-02-06 18:38:50,144] Trial 32 finished with value: 0.8221620902275651 and parameters: {'algorithm': 'brute', 'k_n_neighbors': 9, 'weights': 'uniform', 'p': 2, 'leaf_size': 6}. Best is trial 20 with value: 0.8225613272412723.
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  wa

[I 2023-02-06 19:26:02,908] Trial 42 finished with value: 0.8225613272412723 and parameters: {'algorithm': 'brute', 'k_n_neighbors': 10, 'weights': 'uniform', 'p': 2, 'leaf_size': 6}. Best is trial 20 with value: 0.8225613272412723.
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
[I 2023-02-06 19:31:14,492] Trial 43 finished with value: 0.8225613272412723 and parameters: {'algorithm': 'brute', 'k_n_neighbors': 10, 'weights': 'uniform', 'p': 2, 'leaf_size': 6}. Best is trial 20 with value: 0.8225613272412723.
C:\Users\AL

...

[I 2023-02-06 23:02:03,550] Trial 86 finished with value: 0.8207425808454952 and parameters: {'algorithm': 'kd_tree', 'k_n_neighbors': 8, 'weights': 'uniform', 'p': 2, 'leaf_size': 10}. Best is trial 20 with value: 0.8225613272412723.
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
[I 2023-02-06 23:06:40,872] Trial 87 finished with value: 0.8225613272412723 and parameters: {'algorithm': 'brute', 'k_n_neighbors': 10, 'weights': 'uniform', 'p': 2, 'leaf_size': 6}. Best is trial 20 with value: 0.8225613272412723.
C:\Users\

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
[I 2023-02-06 23:55:59,208] Trial 97 finished with value: 0.8221620902275651 and parameters: {'algorithm': 'brute', 'k_n_neighbors': 9, 'weights': 'uniform', 'p': 2, 'leaf_size': 2}. Best is trial 20 with value: 0.8225613272412723.
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  ret

C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
[I 2023-02-07 00:40:43,665] Trial 108 finished with value: 0.8027325555604844 and parameters: {'algorithm': 'brute', 'k_n_neighbors': 9, 'weights': 'distance', 'p': 1, 'leaf_size': 2}. Best is trial 20 with value: 0.8225613272412723.
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  

[I 2023-02-07 01:22:31,582] Trial 118 finished with value: 0.8225169675730826 and parameters: {'algorithm': 'ball_tree', 'k_n_neighbors': 10, 'weights': 'uniform', 'p': 2, 'leaf_size': 6}. Best is trial 20 with value: 0.8225613272412723.
C:\Users\ALOK  SRIVASTAVA\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
[I 2023-02-07 01:27:11,583] Trial 119 finished with value: 0.8215410548729095 and parameters: {'algorithm': 'brute', 'k_n_neighbors': 6, 'weights': 'uniform', 'p': 2, 'leaf_size': 6}. Best is trial 20 with value: 0.8225613272412723.
C:\Use

Number of finished trials:  128
Best trial:
  Value: 0.8225613272412723
  Params: 
    algorithm: brute
    k_n_neighbors: 10
    weights: uniform
    p: 2
    leaf_size: 2


# xgboost tuning

In [ ]:
"""
Optuna example that optimizes a classifier configuration for cancer dataset
using XGBoost.
In this example, we optimize the validation accuracy of cancer detection
using XGBoost. We optimize both the choice of booster model and its
hyperparameters.
"""

import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def objective(trial):

    dtrain = xgb.DMatrix(X_train_df, label=Y_train_df)
    dvalid = xgb.DMatrix(X_test_df, label=Y_test_df)

    param = {
        "verbosity": 0,
        "objective": "multi:softmax",
        # use exact for small dataset.
        #"tree_method": "exact",
        "num_class" : 10,
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 8, 32),
        "n_estimators": trial.suggest_categorical("n_estimators", [16, 32, 64, 96]),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(Y_test_df, pred_labels)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=128, timeout=360000)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

# Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def rf_objective(trial:optuna.trial.Trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 8, 128),
        'max_depth': trial.suggest_int('max_depth', 4, 16),
        'min_samples_split': trial.suggest_int('min_samples_split', 4, 32),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 16),
    }

    clf = RandomForestClassifier(random_state=42, **params)
    
    clf.fit(X_train_df, Y_train_df)

    preds = clf.predict(X_test_df)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(Y_test_df, pred_labels)
    accuracy

    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(rf_objective, n_trials=128, timeout=360000)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

# Decision Tree

In [18]:
import sklearn
print(sklearn.__version__)

1.0.2


In [19]:
from sklearn.tree import DecisionTreeClassifier


def dt_objective(trial:optuna.trial.Trial):
    params = {
        "splitter" : trial.suggest_categorical("splitter", ["best","random"]),
        "criterion" : trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "max_depth" : trial.suggest_int('max_depth', 4, 16),
        "min_samples_split" : trial.suggest_int('min_samples_split', 4, 32),
        "min_samples_leaf" : trial.suggest_int('min_samples_leaf', 2, 16),
        "min_weight_fraction_leaf" : trial.suggest_float("min_weight_fraction_leaf", 1e-8, 0.5, log=True),
        "min_impurity_decrease" : trial.suggest_float("min_impurity_decrease", 1e-8, 1.0, log=True),
        #"max_features" : trial.suggest_categorical("max_features", [int, float, "auto", "sqrt", "log2"]),
        "max_leaf_nodes" : trial.suggest_int('max_leaf_nodes', 4, 32)
    }

    clf = DecisionTreeClassifier(**params)

    
    clf.fit(X_train_df, Y_train_df)

    preds = clf.predict(X_test_df)
    pred_labels = np.rint(preds)
    accuracy = metrics.accuracy_score(Y_test_df, pred_labels)
    accuracy

    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(dt_objective, n_trials=128, timeout=360000)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-02-07 06:44:59,240] A new study created in memory with name: no-name-aa6ce839-9de9-4df3-b8c5-0259d5748c95
[I 2023-02-07 06:45:01,387] Trial 0 finished with value: 0.7307811737568203 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 16, 'min_samples_split': 22, 'min_samples_leaf': 14, 'min_weight_fraction_leaf': 0.009324116861157052, 'min_impurity_decrease': 0.00017189213377432103, 'max_leaf_nodes': 23}. Best is trial 0 with value: 0.7307811737568203.
[I 2023-02-07 06:45:01,740] Trial 1 finished with value: 0.43073237812181164 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 8, 'min_samples_split': 5, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.006065307658028419, 'min_impurity_decrease': 0.37598691181324434, 'max_leaf_nodes': 7}. Best is trial 0 with value: 0.7307811737568203.
[I 2023-02-07 06:45:03,421] Trial 2 finished with value: 0.6223661447012376 and parameters: {'splitter': 'best', 'criterion': 'entropy', 'max_depth': 5,

[I 2023-02-07 06:45:26,423] Trial 22 finished with value: 0.5328039746262698 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 18, 'min_samples_leaf': 8, 'min_weight_fraction_leaf': 0.00010601042803605302, 'min_impurity_decrease': 0.013358383651778978, 'max_leaf_nodes': 4}. Best is trial 17 with value: 0.778379097724349.
[I 2023-02-07 06:45:26,823] Trial 23 finished with value: 0.43073237812181164 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 12, 'min_samples_split': 28, 'min_samples_leaf': 10, 'min_weight_fraction_leaf': 0.0030989322792233515, 'min_impurity_decrease': 0.7843308439500253, 'max_leaf_nodes': 9}. Best is trial 17 with value: 0.778379097724349.
[I 2023-02-07 06:45:28,388] Trial 24 finished with value: 0.7493235150601073 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 23, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.00021079899101149343, 'min_impuri

[I 2023-02-07 06:45:56,268] Trial 44 finished with value: 0.7152109302222419 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 15, 'min_samples_split': 11, 'min_samples_leaf': 10, 'min_weight_fraction_leaf': 0.005118662114858221, 'min_impurity_decrease': 0.0006042329386952813, 'max_leaf_nodes': 18}. Best is trial 17 with value: 0.778379097724349.
[I 2023-02-07 06:45:58,052] Trial 45 finished with value: 0.5493057711928315 and parameters: {'splitter': 'best', 'criterion': 'entropy', 'max_depth': 16, 'min_samples_split': 21, 'min_samples_leaf': 13, 'min_weight_fraction_leaf': 0.04562786614818407, 'min_impurity_decrease': 0.006862593752719461, 'max_leaf_nodes': 15}. Best is trial 17 with value: 0.778379097724349.
[I 2023-02-07 06:45:59,192] Trial 46 finished with value: 0.778379097724349 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 13, 'min_samples_split': 26, 'min_samples_leaf': 11, 'min_weight_fraction_leaf': 0.0010622395977792818, 'min_imp

[I 2023-02-07 06:46:22,348] Trial 66 finished with value: 0.6677017255910925 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 4, 'min_samples_split': 25, 'min_samples_leaf': 10, 'min_weight_fraction_leaf': 0.0003848618060397913, 'min_impurity_decrease': 5.79131091950365e-06, 'max_leaf_nodes': 12}. Best is trial 17 with value: 0.778379097724349.
[I 2023-02-07 06:46:23,693] Trial 67 finished with value: 0.7493235150601073 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 29, 'min_samples_leaf': 9, 'min_weight_fraction_leaf': 0.00013307616350939714, 'min_impurity_decrease': 1.3683264751365257e-06, 'max_leaf_nodes': 12}. Best is trial 17 with value: 0.778379097724349.
[I 2023-02-07 06:46:24,940] Trial 68 finished with value: 0.778379097724349 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 6, 'min_samples_split': 24, 'min_samples_leaf': 11, 'min_weight_fraction_leaf': 0.0002651224445044075, 'min_impur

[I 2023-02-07 06:46:51,154] Trial 88 finished with value: 0.5541853346936965 and parameters: {'splitter': 'random', 'criterion': 'gini', 'max_depth': 4, 'min_samples_split': 31, 'min_samples_leaf': 11, 'min_weight_fraction_leaf': 0.004027549820281127, 'min_impurity_decrease': 1.4349259518063394e-07, 'max_leaf_nodes': 31}. Best is trial 78 with value: 0.7895133744399592.
[I 2023-02-07 06:46:52,967] Trial 89 finished with value: 0.7872066716940957 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 32, 'min_samples_leaf': 10, 'min_weight_fraction_leaf': 0.0025580307797346768, 'min_impurity_decrease': 7.796460299475847e-06, 'max_leaf_nodes': 29}. Best is trial 78 with value: 0.7895133744399592.
[I 2023-02-07 06:46:54,453] Trial 90 finished with value: 0.7874284700350441 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 32, 'min_samples_leaf': 10, 'min_weight_fraction_leaf': 0.002683533804910189, 'min_i

[I 2023-02-07 06:47:21,533] Trial 110 finished with value: 0.7865856363394402 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 32, 'min_samples_leaf': 4, 'min_weight_fraction_leaf': 0.001523923286326509, 'min_impurity_decrease': 1.5296720699074194e-05, 'max_leaf_nodes': 27}. Best is trial 78 with value: 0.7895133744399592.
[I 2023-02-07 06:47:23,166] Trial 111 finished with value: 0.7865856363394402 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 31, 'min_samples_leaf': 10, 'min_weight_fraction_leaf': 0.0017113377217278301, 'min_impurity_decrease': 8.705838291900582e-06, 'max_leaf_nodes': 29}. Best is trial 78 with value: 0.7895133744399592.
[I 2023-02-07 06:47:24,713] Trial 112 finished with value: 0.783258661225214 and parameters: {'splitter': 'best', 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 31, 'min_samples_leaf': 10, 'min_weight_fraction_leaf': 0.0030786520048421487, 'min_i

Number of finished trials:  128
Best trial:
  Value: 0.7895133744399592
  Params: 
    splitter: best
    criterion: gini
    max_depth: 5
    min_samples_split: 31
    min_samples_leaf: 9
    min_weight_fraction_leaf: 6.200532790768602e-05
    min_impurity_decrease: 4.0913217737603715e-06
    max_leaf_nodes: 30


# SVM

In [20]:
from sklearn.svm import SVC


def svm_objective(trial):
    # C
    svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
    # kernel
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf'])
    # SVC
    clf = SVC(C=svc_c, kernel=kernel)
    clf.fit(X_train_df, Y_train_df)

    preds = clf.predict(X_test_df)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(Y_test_df, pred_labels)
    accuracy

    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(svm_objective, n_trials=128, timeout=36000000)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-02-07 06:47:46,043] A new study created in memory with name: no-name-b24ab5f1-34d8-4655-b6bb-eb2af1e29410
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-02-07 07:04:04,771] Trial 0 finished with value: 0.8136893936033358 and parameters: {'C': 1.3214080241082582, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8136893936033358.
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning

C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-02-07 08:57:04,014] Trial 10 finished with value: 0.7831699418888347 and parameters: {'C': 94.89309293642287, 'kernel': 'linear'}. Best is trial 1 with value: 0.8488666104777536.
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.

[I 2023-02-07 10:58:03,764] Trial 20 finished with value: 0.783125582220645 and parameters: {'C': 57.91729897042338, 'kernel': 'linear'}. Best is trial 13 with value: 0.8513951115645655.
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-02-07 11:08:25,133] Trial 21 finished with value: 0.8509958745508583 and parameters: {'C': 91.90690374218131, 'kernel': 'poly'}. Best is trial 13 with value: 0.8513951115645655.
C:\Users\ALOK  S

C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-02-07 12:18:10,344] Trial 31 finished with value: 0.8511289535554274 and parameters: {'C': 93.50578969559342, 'kernel': 'poly'}. Best is trial 13 with value: 0.8513951115645655.
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.c

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-02-07 14:27:49,810] Trial 41 finished with value: 0.8515281905691345 and parameters: {'C': 99.71151319562165, 'kernel': 'poly'}. Best is trial 41 with value: 0.8515281905691345.
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, )

...

C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-02-07 22:54:25,773] Trial 93 finished with value: 0.8505966375371512 and parameters: {'C': 88.24748970679333, 'kernel': 'poly'}. Best is trial 41 with value: 0.8515281905691345.
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.c

[I 2023-02-08 00:18:21,474] Trial 103 finished with value: 0.8505079182007719 and parameters: {'C': 84.92816979640865, 'kernel': 'poly'}. Best is trial 41 with value: 0.8515281905691345.
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-02-08 00:25:41,086] Trial 104 finished with value: 0.8514394712327552 and parameters: {'C': 99.56812782477611, 'kernel': 'poly'}. Best is trial 41 with value: 0.8515281905691345.
C:\Users\ALOK  

C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-02-08 01:37:30,214] Trial 114 finished with value: 0.8507297165417202 and parameters: {'C': 90.72970689525542, 'kernel': 'poly'}. Best is trial 41 with value: 0.8515281905691345.
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.

[I 2023-02-08 02:48:33,460] Trial 124 finished with value: 0.8512620325599964 and parameters: {'C': 98.20873217664943, 'kernel': 'poly'}. Best is trial 41 with value: 0.8515281905691345.
C:\Users\ALOK  SRIVASTAVA\AppData\Local\Temp\ipykernel_11560\1089354057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  svc_c = trial.suggest_loguniform('C', 1e0, 1e2)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-02-08 02:55:41,656] Trial 125 finished with value: 0.8514394712327552 and parameters: {'C': 99.3869084054345, 'kernel': 'poly'}. Best is trial 41 with value: 0.8515281905691345.
C:\Users\ALOK  S

Number of finished trials:  128
Best trial:
  Value: 0.8515281905691345
  Params: 
    C: 99.71151319562165
    kernel: poly
